<a href="https://colab.research.google.com/github/jmonsa13/Scrapping/blob/main/Auto_scrap_Lowes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main Code

In [ ]:
# Python project for pricing scripping of different sites
# Creado por: Juan Monsalvo

# FOR Lowes
# ----------------------------------------------------------------------------------------------------------------------
# Libraries import
# ----------------------------------------------------------------------------------------------------------------------
import datetime
import numpy as np
import pandas as pd
import re
import json
import requests
import time
import os

from google.colab import drive
drive.mount('/drive')

# ----------------------------------------------------------------------------------------------------------------------
# Configuration and Global Variables
# ----------------------------------------------------------------------------------------------------------------------
# Header for beautiful soup
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0",
    'Cookie': 'sn=2348;'
}
cookies = {"sn":"2348", 'region' : 'east'} # Central Charlotte 2348

# Path definition of the .csv file
fecha = datetime.datetime.today()

# Sheet Mansfield
sheet_name_mansfield = 'Mansfield'
output_path_toilet_mansfield = '/drive/MyDrive/00_SCRAPPING/XX_Data/Lowes_Mansfield-' + str(fecha.year) + '_' + \
                               str(fecha.month) + '.csv'

# Sheet Competitors
sheet_name_competitor = 'Competitors'
output_path_toilet_competitor = '/drive/MyDrive/00_SCRAPPING/XX_Data/Lowes_Competitors-' + str(fecha.year) + '_' + \
                                str(fecha.month) + '.csv'

# Path for loading the URL sites
url_path_toilet = '/drive/MyDrive/00_SCRAPPING/XX_URL/Lowes_Database.xlsx'

product_path =  [[url_path_toilet, output_path_toilet_mansfield, sheet_name_mansfield],
                 [url_path_toilet, output_path_toilet_competitor, sheet_name_competitor]]

# Number of retry
NUM_RETRIES = 5

# Waiting time between request
delays = [1, 4, 8, 2, 5, 3]

# ----------------------------------------------------------------------------------------------------------------------
# Function Definition
# ----------------------------------------------------------------------------------------------------------------------
def lowes_data(elem, data):
    """
    Programa que toma la información general de una pagina de producto del market place de Ferguson
    """
    item_id = elem['Link'].split("/")[-1]
    # print(json.dumps(data["productDetails"][item_id], indent=4, sort_keys=True))

    # Collecting the name and product type
    brand_name = elem["Fabricante"]
    product_name = elem["Description"]
    product_subcategory = elem["Subcategory"]
    product_format = elem["Tipo"]
    linea_name = elem["Linea"]
    price_type = elem["Price Type"]

    # Collecting the price
    try:
      price_clean = float(data["productDetails"][item_id]["price"]["analyticsData"]["sellingPrice"])
    except:
      price_clean = ""

    try:
      was_price_clean = float(data["productDetails"][item_id]["price"]["analyticsData"]["wasPrice"])
    except:
      was_price_clean = ""

    try:
      offert_price = float(data["productDetails"][item_id]["offerPromotions"]["offerPrice"])
    except:
      offert_price = ""

    # print("PromoMsg:", data["productDetails"][item_id]["offerPromotions"]["displayPromoMsg"])
    # print("Currency:", data["productDetails"][item_id]["price"]["newPrice"]["price"]["currency"])

    # Collecting the sku of product
    sku_ref = data["productDetails"][item_id]["product"]["omniItemId"]
    item_ref = data["productDetails"][item_id]["product"]["itemNumber"]
    model_ref = data["productDetails"][item_id]["product"]["modelId"]

    # Collecting the image
    aux_url_img = data["productDetails"][item_id]["product"]["imageUrls"][0]["value"]
    url_img = 'https://mobileimages.lowes.com' + aux_url_img

    # Stock
    try:
      aux_stock = int(data["productDetails"][item_id]["itemInventory"]["totalAvailableQty"])
      if aux_stock >= 0:
        stock = "Si"
      else:
        stock = "No"
    except:
      stock = "No Disponible"
    # ------------------------------------------------------------------------------------------------------------------
    # Message display
    print("Recopilando la información de {}".format(elem["Link"]))
    print("La marca es la: {}".format(brand_name))
    print("El sanitario es el: {}".format(product_name))
    print("El tipo de producto es un: {}".format(product_format))
    print("El sku es el: {}".format(sku_ref))
    print("El item es el: {}".format(item_ref))
    print("El model es el: {}".format(model_ref))
    print("El precio actual es: {} USD".format(price_clean))
    print("El precio anterior es: {} USD".format(was_price_clean))
    print("El precio con descuento especial es: {} USD".format(offert_price))
    print("Inventario:{}".format(stock))
    print(url_img)
    print("\n")
    # ------------------------------------------------------------------------------------------------------------------

    # Appending the item in a list
    information = [datetime.datetime.today().date(), brand_name, elem["Sku"],
                   linea_name, product_subcategory, product_format, elem["Rough in"], elem["Bowl Height"], elem["Asiento"],
                   elem["Capacidad"], product_name, price_type, price_clean, was_price_clean, offert_price,
                   "USD", "lowes.com", "Si", elem["Link"], url_img]

    return information


# ----------------------------------------------------------------------------------------------------------------------
# Main code
# ----------------------------------------------------------------------------------------------------------------------
# Scrapping the information of every url
for product_type, output_path, sheet_name  in product_path:
    # Reading .xlsx file with url list
    file_df = pd.read_excel(product_type, sheet_name=sheet_name)

    # Keeping just the row with links
    product_df = file_df[file_df['Link'].notna()]

    # Scrapping the information of every product
    datos = []
    for index, elem in product_df.iterrows():
      for i in range(NUM_RETRIES):
          # Random Wait between request
          delay = np.random.choice(delays)
          time.sleep(delay)
          try:
              response = requests.get(elem["Link"], headers=headers, cookies=cookies)
              # response = requests.get(elem["Link"], headers=headers, meta=meta)


              print("Intento numero {}".format(i))
              print(response)
              if response.status_code in [200, 404]:
                  # escape for loop if the API returns a successful response
                  break
          except requests.exceptions.ConnectionError:
              response = ''
              print("ConnectionError")

      # parse data if 200 status code (successful response)
      if response.status_code == 200:
          # Parse data with json
          t = response.text
          data = re.search(r"window\['__PRELOADED_STATE__'\] = (\{.*?\})<", t)
          data = json.loads(data.group(1))
          
          # Uncomment to print all data:
          # print(json.dumps(data, indent=4))

          # Getting the information from the website
          toilet_information = lowes_data(elem, data)
          datos.append(toilet_information)

    # Creating the dataframe
    df = pd.DataFrame(datos, columns=["Fecha", "Fabricante", "SKU", "Linea", 'Subcategoria', "Tipo", "Rough_In",
                                     "Bowl_Height", "Asiento", "Capacidad_(Gpl)", "Producto",
                                     "Price_Type", "Precio_Consumidor", "Precio_Anterior", "Precio_Promocion", "Moneda",
                                     "Market_Place", "Stock", "URL", "Image_url"])
    
    # Saving the file in a .csv file
    df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

Mounted at /drive
Intento numero 0
<Response [200]>
Recopilando la información de https://www.lowes.com/pd/Mansfield-Alto-174-1-28-Round-SmartHeight-8482-Toilet-Combination/5002186849
La marca es la: Mansfield
El sanitario es el: Alto CTK 12 ADA RF 1,28 gpf
El tipo de producto es un: CTK
El sku es el: 5002186849
El item es el: 4144313
El model es el: 117-3173
El precio actual es: 209.0 USD
El precio anterior es:  USD
El precio con descuento especial es: 198.55 USD
Inventario:Si
https://mobileimages.lowes.com/productimages/2015302a-710d-4a0a-82c0-b499ddf87b1e/44125209.jpg


Intento numero 0
<Response [200]>
Recopilando la información de https://www.lowes.com/pd/Mansfield-Alto-White-WaterSense-Elongated-Comfort-Height-2-Piece-Vitreous-China-Toilet-12-in-Rough-In-Size-ADA-Compliant/1002970010
La marca es la: Mansfield
El sanitario es el: Alto CTK 12 ADA RF 1,28 gpf
El tipo de producto es un: CTK
El sku es el: 1002970010
El item es el: 2412589
El model es el: 5117CTK
El precio actual es: 1

## Testing API

In [ ]:
import requests, json
from urllib.parse import urlencode


# Tell scraper to use Scraper API as the proxy
API_KEY = 'd9d061ea3ad19b7f0eb56c83f416ba06'

product_id = 5002186849 # Mansfield  Alto White Round Chair Height 2-piece
store_location_id = 2348  # Central Charlotte lowe's

url = f"https://www.lowes.com/PricingServices/price/balance?productId={product_id}&storeNumber={store_location_id}" 

params = {'api_key': API_KEY, 'url': url, "country_code": "us"}

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0",
}


#r = requests.get(url, headers=headers, timeout=5)
r = requests.get('http://api.scraperapi.com/', params=urlencode(params))

print("return code:", r)
print("content length:", len(r.content))
print(json.dumps(json.loads(r.text), indent=True))

In [ ]:
import requests

# set up the request parameters
params = {
  'api_key': 'demo',
  'item_id': '5002186849'
}

# make the http GET request to Backyard API
api_result = requests.get('https://api.backyardapi.com/request', params)

# print the JSON response from Backyard API
print(json.dumps(api_result.json()))


## Testing new way SCRAPY

In [ ]:
# Python project for pricing scripping of different sites
# Creado por: Juan Monsalvo

# FOR Lowes
# ----------------------------------------------------------------------------------------------------------------------
# Libraries import
# ----------------------------------------------------------------------------------------------------------------------
import scrapy
from scrapy.http import Request
import requests
import pandas as pd


class LowesSpider(scrapy.Spider):
    name = 'lowes'
    def start_requests(self):

        start_urls = ['https://www.lowes.com/pd/Mansfield-Alto-174-1-28-Round-SmartHeight-8482-Toilet-Combination/5002186849']

        for url in start_urls:
            yield Request(url,
                        headers={'Cookie': 'sn=2348;'}, #Preset a location
                        meta={'dont_merge_cookies': True, #Allows location cookie to get through
                        'url':url}) #Using to get the product SKU

    def parse(self, response):
        saleprice = response.css('span[itemprop="price"]::attr(content)').extract_first()
        wasprice_text = response.css('span.art-pd-wasPriceLbl::text').extract_first()
        if "$" in wasprice_text:
            fullprice =  wasprice_text.split("$")[-1]

        item["productPrice"] = offered_price
        yield item

# Connection retriving

In [ ]:
!curl ipecho.net/plain

34.135.43.220

In [ ]:
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0",
}

rr = requests.get('http://www.showmyip.com', headers=headers)
soup = BeautifulSoup(rr.content, "html.parser")

ip_d = soup.find('h2', {'id':'ipv4'}).text
ipv4_c = soup.find('table', class_ = 'iptab').find_all('td')[1].text
country_c = soup.find('table', class_ = 'iptab').find_all('td')[5].text
region_c = soup.find('table', class_ = 'iptab').find_all('td')[7].text
city_c = soup.find('table', class_ = 'iptab').find_all('td')[9].text
organization_c = soup.find('table', class_ = 'iptab').find_all('td')[17].text
AS_number_and_name_c = soup.find('table', class_ = 'iptab').find_all('td')[19].text

datos = {'ipv4':ipv4_c,
      'country':country_c,
      'region':region_c,
      'city':city_c,
      'organization':organization_c,
      'as_number_as_name':AS_number_and_name_c   
}

print(datos)

{'ipv4': '34.75.29.50', 'country': 'United States', 'region': 'South Carolina', 'city': 'North Charleston', 'organization': 'Google Cloud (us-east1)', 'as_number_as_name': 'AS396982 Google LLC'}


# Pip Install

In [ ]:
!pip install scrapy
!pip install selenium
!pip install selenium-wire
!pip install webdriver-manager
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
!pip install scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 264 kB 5.6 MB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
     |████████████████████████████████| 3.1 MB 31.8 MB/s 
     |████████████████████████████████| 93 kB 2.2 MB/s 
     |████████████████████████████████| 251 kB 40.1 MB/s 
     |████████████████████████████████| 4.0 MB 44.4 MB/s 
     |████████████████████████████████| 74 kB 3.2 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11958 sha256=583d8fec4c71659e2748a4ead02856c6dd0abaa0cd8c24eb6401caa10b0570a3
  Stored in directory: /root/.cache/pip/wheels/c9/d6/6a/de198d890277cde60ca3dbebe7ae592d3b381c7d9bb2455f4d
Successfully built PyDispatcher
